In [1]:
import numpy as np
import scipy.io, scipy.io.wavfile
import oct2py
import librosa as rosa
import tqdm
# Initialization of Matplotlib
from matplotlib import pyplot as plt
plt.style.use("solarized-light")

In [2]:
class Constants(object):
    # Seed of Generating Random Nums
    RANDOM_STATE = 39
    # Frequency of Middle C
    FREQ_MIDDLE_C = 261.6255653 # <=> 440 * 2 ** (-9 / 12) = 440 / 2 ** .75
    # Frequency of Sampling Rate
    FREQ_SAMPLING_RATE = 44100
    # Num of Channels
    NUM_CHANNELS = 1
    # Path of `matlab-midi` Files
    PATH_MATLAB_MIDI_SRC = "./lib/matlab-midi/src/"
    # Path of the MIDI File
    PATH_MIDI_FILE = "./STAGE_OF_SEKAI.mid"
    # Length of Noises' Chunk
    # (which Determines the Tone, as the Content of each Term.)
    LEN_NOISECHUNK = 4096
    # Coefficients
    COEF_DELAY = 1.28
    COEF_SPEED = 192 / 120
# Setting the Random State
np.random.seed(Constants.RANDOM_STATE)

In [3]:
# 一块基础噪声, 其作为一个周期的内容决定着音色,
# 保证其长度 > (采样率 / 频率) + 1, 足够即可
NOISECHUNK = np.random.randn(Constants.LEN_NOISECHUNK, Constants.NUM_CHANNELS)
NOISECHUNK -= NOISECHUNK.mean()
NOISECHUNK /= NOISECHUNK.std()

采样频率 $ F_{samlping} = $ `44100`

***

如若需要合成的声音,

频率为 $ f $ `(Hz)`,

则其所对应的

周期 $ T = \frac{1}f $ `(s)`,

因此一块噪声的长度 $ p = F_{sampling} \cdot T = \frac{F_{sampling}}f $,

***

注:
* *`Karplus-Strong` 算法在 `[300, 1000] (Hz)` 范围内表现较好。*

In [4]:
def midi2freq(pitch:int) -> float:
    # MIDI 音高 -> 频率值
    # <=> (440 / 32) * (2 ** (x - 9) / 12)
    return 55 * (2 ** (pitch / 12 - 2.75))

In [5]:
def freq2length(freq:float) -> int:
    # 频率值 -> 振动一个"周期"的长度
    return np.ceil(Constants.FREQ_SAMPLING_RATE / freq).astype(int)

In [6]:
class Fourier_Sequence(object):
    """
    >>> arr = [0] + [1] * 127 + [0] + [-1] * 127
    >>> fourier = Fourier_Sequence(arr, 39)
    >>> np.array([ fourier.fval(x + .5) for x in range(len(arr)) ])
    """
    def __init__(self, arr:np.ndarray, numiter:int=6):
        self.__arr = np.array(arr).flatten()
        self.__numiter = numiter
        # Parsing Parameters
        self.__term = len(self.arr)
        self.__params = np.zeros((self.numiter, 2)) # [ (a1, b1), (a2, b2), ..., (am, bm) ]
        self.__tricoef = np.array([ ((k + 1) * 2 * np.pi / self.__term) for k in range(self.__numiter) ])
        for k in range(self.__numiter):
            self.__params[k, : ] += sum([
                self.__arr[l] * np.array([
                    np.cos((l + .5) * self.__tricoef[k]),
                    np.sin((l + .5) * self.__tricoef[k]),
                    ])
                for l in range(self.__term)
                ])
    @property
    def arr(self):
        return self.__arr
    @property
    def numiter(self):
        return self.__numiter
    @property
    def parameters(self):
        return self.__params
    # Value Estimated
    def fval(self, x):
        return sum([
            (self.__params[k, : ] @ [ np.cos(x * self.__tricoef[k]), np.sin(x * self.__tricoef[k]) ])
            for k in range(self.__numiter)
            ])

In [7]:
def noise2period(length:int, method:int=0) -> np.ndarray:
    # 把噪声块压缩为需要的长度, 该长度由前文提到的"周期"所决定
    assert 0 < length <= Constants.LEN_NOISECHUNK
    assert method in [ 0, 1 ]
    res = np.zeros((length, Constants.NUM_CHANNELS))
    if method == 0:
        res += NOISECHUNK.copy()[ : length , : ]
    elif method == 1:
        res += NOISECHUNK.copy()[ :: Constants.LEN_NOISECHUNK // length , : ][ : length , : ]
    elif method == 2:
        pass
    return res

In [8]:
def karplus_strong(freq = Constants.FREQ_MIDDLE_C, ti = 3.9, rate_decay:float=0., rate_update:float=.5):
    noisechunk = noise2period(freq2length(freq), 1)
    len_period = noisechunk.shape[0]
    len_output = np.round(Constants.FREQ_SAMPLING_RATE * ti).astype(int)
    num_periods = int(len_output / len_period) - 1
    new_indices = [noisechunk.shape[0] - 1] + list(range(noisechunk.shape[0] - 1))
    res = np.zeros((len_output, Constants.NUM_CHANNELS))
    for k in range(num_periods):
        noisechunk = noisechunk * (1 - rate_update) + noisechunk[new_indices, : ] * rate_update
        res[ len_period * k : len_period * (k + 1) , : ] += noisechunk
    return res

In [9]:
def parsemidi(filepath:str=Constants.PATH_MIDI_FILE, keyname:str="arr") -> np.ndarray:
    # Loading the MIDI Library `matlab-midi`
    oct2py.octave.eval("addpath(\"%s\");" % Constants.PATH_MATLAB_MIDI_SRC)
    # Indices:
    #     0: Pitch in MIDI;
    #     1: Time to Begin;
    #     2: Time to End;
    oct2py.octave.eval("%s = midiInfo(readmidi(\"%s\"), 0)( : , [3, 5, 6]);" % (keyname, filepath))
    # https://stackoverflow.com/questions/45525233/loading-mat-and-m-files-with-loadmat-in-python/
    oct2py.octave.eval("save \"%s.mat\" %s -v7" % (filepath, keyname))
    # Loading the `.mat` File Generated above, which Contains the Tensor of MIDI
    matfile = scipy.io.loadmat(filepath + ".mat")[keyname]
    # Length of a Tensor of a 1-second-long Note
    len_sec = Constants.FREQ_SAMPLING_RATE / Constants.COEF_SPEED
    # Length of the whole Song
    len_output = np.ceil((matfile[ -1, 2 ] + 3.9) * len_sec).astype(int) # 多加一点是为了防止音符渲染出的音频溢出总音频长度
    res = np.zeros((len_output, Constants.NUM_CHANNELS))
    for k in tqdm.trange(matfile.shape[0]):
        freq_midi    = matfile[k, 0]
        time_starter = matfile[k, 1]
        time_ender   = matfile[k, 2]
        index_starter = np.round(time_starter * len_sec).astype(int) # 加入速度系数
        # Newnote Generated by Algorithm "Karplus-Strong"
        newnote = karplus_strong(midi2freq(freq_midi), (time_ender - time_starter) * Constants.COEF_DELAY / Constants.COEF_SPEED)
        res[ index_starter : index_starter + newnote.shape[0] , : ] += newnote
    res /= np.abs(res).max()
    print("Notes Rendering Finished!!")
    return res

In [10]:
output = parsemidi()
if True:
    scipy.io.wavfile.write(Constants.PATH_MIDI_FILE + ".wav", Constants.FREQ_SAMPLING_RATE, output)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 207/207 [00:02<00:00, 95.44it/s]

Notes Rendering Finished!!
